***Question 3 -
Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%
Note -***
1. Code comments should be given for proper code understanding.

2. Implement in both PyTorch and Tensorflow respectively

### PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Set random seed for reproducibility
torch.manual_seed(42)

In [3]:
# Define transformations to apply to the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
# Load MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:03<00:00, 2876289.85it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 1312995.96it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 3518839.21it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 2272248.18it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [5]:
# Define dataloaders for training and testing
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

In [6]:
# Check if CUDA is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Define a simple CNN with less than 10000 parameters
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 7 * 7, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
# Instantiate the CNN model
net = Net().to(device)

In [9]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [10]:
# Training loop
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"[{epoch + 1}] Loss: {running_loss / len(trainloader)}")

[1] Loss: 0.3854492844771475
[2] Loss: 0.09480279464044296
[3] Loss: 0.06730267057008645
[4] Loss: 0.05138291248769712
[5] Loss: 0.04329479915369898
[6] Loss: 0.03697613977702029
[7] Loss: 0.03122086300135358
[8] Loss: 0.028642123607523454
[9] Loss: 0.02316238466915347
[10] Loss: 0.021401189398819777


In [11]:
# Evaluate the model on the test set
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

Validation Accuracy: 98.94%


## TensorFlow 

In [12]:
import tensorflow as tf
from tensorflow.keras import layers

In [13]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [14]:
# Normalize pixel values between -1 and 1
x_train = (x_train / 255.0) * 2 - 1
x_test = (x_test / 255.0) * 2 - 1

In [15]:
# Add a channel dimension to the images
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [16]:
# Create TensorFlow Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128)

In [17]:
# Define a simple CNN with less than 10000 parameters
model = tf.keras.Sequential([
    layers.Conv2D(8, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

In [18]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
# Train the model
model.fit(train_dataset, epochs=10)

Epoch 1/10
469/469 [==============================] - 48s 87ms/step - loss: 0.3592 - accuracy: 0.8962
Epoch 2/10
469/469 [==============================] - 38s 81ms/step - loss: 0.0974 - accuracy: 0.9705
Epoch 3/10
469/469 [==============================] - 42s 89ms/step - loss: 0.0670 - accuracy: 0.9798
Epoch 4/10
469/469 [==============================] - 40s 86ms/step - loss: 0.0528 - accuracy: 0.9840
Epoch 5/10
469/469 [==============================] - 37s 80ms/step - loss: 0.0439 - accuracy: 0.9869
Epoch 6/10
469/469 [==============================] - 45s 95ms/step - loss: 0.0368 - accuracy: 0.9891
Epoch 7/10
469/469 [==============================] - 51s 110ms/step - loss: 0.0312 - accuracy: 0.9912
Epoch 8/10
469/469 [==============================] - 44s 94ms/step - loss: 0.0265 - accuracy: 0.9925
Epoch 9/10
469/469 [==============================] - 43s 92ms/step - loss: 0.0225 - accuracy: 0.9939
Epoch 10/10
469/469 [==============================] - 58s 123ms/step - loss: 0.0

In [20]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Validation Accuracy: {test_accuracy * 100:.2f}%")

79/79 [==============================] - 5s 30ms/step - loss: 0.0511 - accuracy: 0.9856
Validation Accuracy: 98.56%
